In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier #Random Forest is a series of decision trees but each decision tree has different parameters

matches = pd.read_csv('matches.csv', index_col = 0)
matches.head()
matches.shape

#Investigate missing data
matches['team'].value_counts()
matches[matches['team'] == 'Liverpool']
matches['round'].value_counts()

#Cleaning data for Machine Learning
matches.dtypes #ML only can work with numeric data 
matches["date"] = pd.to_datetime(matches["date"])
matches["target"] = (matches["result"] == "W").astype("int")
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches

#Creating initial Machine Learning model
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1) #more n estimators takes longer to run but perhaps more accurate, larger min_samples split less likely to overfit
#but less accurate, random state setsthe seed for the random number generator, ensuring reproducibility of results.
train = matches[matches["date"] < '2023-01-01'] #train data
test = matches[matches["date"] > '2023-01-01'] #test data
predictors = ["venue_code", "opp_code", "hour", "day_code"] 
rf.fit(train[predictors], train["target"]) #first param is the features the model will use to make predictions and the second param is what we are predicting
preds = rf.predict(test[predictors]) #Uses the trained Random Forest model (rf) to make predictions for the test data (test).

error = accuracy_score(test["target"], preds) #Computes the accuracy of the predictions made by the model.
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds)) #Creates a DataFrame that combines the actual labels and predicted labels for easier comparison.
pd.crosstab(index=combined["actual"], columns=combined["predicted"]) #create two way table w crosstab

precision_score(test["target"], preds) #It measures how many of the predicted positive labels are actually correct.

#Improving precision with rolling averages
grouped_matches = matches.groupby("team") 

def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean() #closed=left ensures rolling window does not include the current match (only the previous 3 matches are considered)
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols) #drop any missing values like first 2 GW
    return group
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]


matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

#Retraining machine model 
def make_predictions(data, predictors):
    train = data[data["date"] < '2023-01-01']
    test = data[data["date"] > '2023-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision 

combined, precision = make_predictions(matches_rolling, predictors + new_cols)


combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True) #left_index and right_index = true merges base off same index





/var/folders/jh/7wm9ksm12d5cm5v95w2kwj740000gn/T/ipykernel_10044/1419438712.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


0.5158371040723982